In [7]:
# Gym stuff
import gym
import gym_anytrading
# Stable baselines - rl stuff
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, PPO

# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from trader import trader_agent
import cryptomanager as cm

from config import LOG_PATH, MODEL_PATH
from datetime import datetime
import os
test = trader_agent()

1


In [8]:
df = cm.historical_data_df(days_delta=10000)
print(df.dtypes)
df

1995-06-19 00:00:00
date     datetime64[ns, UTC]
Open                 float64
High                 float64
Low                  float64
Close                float64
dtype: object


,date,Open,High,Low,Close
0,2020-04-08 08:09:00+00:00,7314.0,7314.0,7314.0,7314.0
1,2020-04-14 06:31:00+00:00,7101.5,7101.5,7101.5,7101.5
2,2020-04-14 07:55:00+00:00,7100.0,7100.0,7100.0,7100.0
3,2020-04-22 23:29:00+00:00,7118.5,7118.5,7118.5,7118.5
4,2020-04-28 12:28:00+00:00,7748.5,7748.5,7748.5,7748.5
...,...,...,...,...,...
581994,2022-11-05 01:03:00+00:00,21410.0,21410.0,21400.0,21400.0
581995,2022-11-05 01:04:00+00:00,21400.0,21406.0,21376.0,21376.0
581996,2022-11-05 01:05:00+00:00,21384.0,21384.0,21347.0,21347.0
581997,2022-11-05 01:06:00+00:00,21347.0,21347.0,21335.0,21335.0


In [9]:
window_size = 100
lower_bound = len(df) - window_size
upper_bound = len(df)
# df = pd.read_csv("data/gmedata.csv")
env = gym.make('stocks-v0', df=df, frame_bound=(lower_bound,upper_bound), window_size=window_size)
df[lower_bound:upper_bound]

,date,Open,High,Low,Close
581899,2022-11-04 23:23:00+00:00,21159.0,21159.0,21159.0,21159.0
581900,2022-11-04 23:24:00+00:00,21159.0,21159.0,21159.0,21159.0
581901,2022-11-04 23:25:00+00:00,21160.0,21160.0,21160.0,21160.0
581902,2022-11-04 23:26:00+00:00,21157.0,21164.0,21157.0,21164.0
581903,2022-11-04 23:27:00+00:00,21165.0,21165.0,21165.0,21165.0
...,...,...,...,...,...
581994,2022-11-05 01:03:00+00:00,21410.0,21410.0,21400.0,21400.0
581995,2022-11-05 01:04:00+00:00,21400.0,21406.0,21376.0,21376.0
581996,2022-11-05 01:05:00+00:00,21384.0,21384.0,21347.0,21347.0
581997,2022-11-05 01:06:00+00:00,21347.0,21347.0,21335.0,21335.0


In [10]:
# # Examining the model

# state = env.reset()
# while True: 
#     # add a bit of randomness
#     # action = env.action_space.sample()
#     # n_state, reward, done, info = env.step(action)
#     if done: 
#         print("info", info)
#         break
        
# plt.figure(figsize=(15,6))
# plt.cla()
# env.render_all()
# plt.show()

In [11]:
# !pip install finta

In [12]:
from gym_anytrading.envs import StocksEnv, ForexEnv
from finta import TA

In [13]:
# df['OBV'] = TA.OBV(df)
df['EMA'] = TA.EMA(df, 200)
df['RSI'] = TA.RSI(df)
df.fillna(0, inplace=True)
df = df.join(TA.MACD(df))
df

,date,Open,High,Low,Close,EMA,RSI,MACD,SIGNAL
0,2020-04-08 08:09:00+00:00,7314.0,7314.0,7314.0,7314.0,7314.000000,0.000000,0.000000,0.000000
1,2020-04-14 06:31:00+00:00,7101.5,7101.5,7101.5,7101.5,7207.218750,0.000000,-4.767628,-2.648682
2,2020-04-14 07:55:00+00:00,7100.0,7100.0,7100.0,7100.0,7171.121178,0.000000,-6.121420,-4.071935
3,2020-04-22 23:29:00+00:00,7118.5,7118.5,7118.5,7118.5,7157.767900,9.107921,-5.919194,-4.697701
4,2020-04-28 12:28:00+00:00,7748.5,7748.5,7748.5,7748.5,7278.289003,79.058101,19.384455,2.466196
...,...,...,...,...,...,...,...,...,...
581994,2022-11-05 01:03:00+00:00,21410.0,21410.0,21400.0,21400.0,21186.357662,63.437409,40.123224,48.396812
581995,2022-11-05 01:04:00+00:00,21400.0,21406.0,21376.0,21376.0,21188.244650,55.140704,35.209718,45.759393
581996,2022-11-05 01:05:00+00:00,21384.0,21384.0,21347.0,21347.0,21189.824306,47.121192,28.645464,42.336607
581997,2022-11-05 01:06:00+00:00,21347.0,21347.0,21335.0,21335.0,21191.268840,44.253136,22.218822,38.313050


In [14]:
df.head(15)

,date,Open,High,Low,Close,EMA,RSI,MACD,SIGNAL
0,2020-04-08 08:09:00+00:00,7314.0,7314.0,7314.0,7314.0,7314.000000,0.000000,0.000000,0.000000
1,2020-04-14 06:31:00+00:00,7101.5,7101.5,7101.5,7101.5,7207.218750,0.000000,-4.767628,-2.648682
2,2020-04-14 07:55:00+00:00,7100.0,7100.0,7100.0,7100.0,7171.121178,0.000000,-6.121420,-4.071935
3,2020-04-22 23:29:00+00:00,7118.5,7118.5,7118.5,7118.5,7157.767900,9.107921,-5.919194,-4.697701
4,2020-04-28 12:28:00+00:00,7748.5,7748.5,7748.5,7748.5,7278.289003,79.058101,19.384455,2.466196
5,2020-04-29 03:18:00+00:00,7836.0,7836.0,7836.0,7836.0,7373.580045,81.219889,37.972569,12.090397
6,2020-04-30 12:48:00+00:00,8829.0,8829.0,8829.0,8829.0,7587.786410,91.696109,94.223816,32.876174
7,2020-04-30 12:56:00+00:00,8734.5,8734.5,8734.5,8734.5,7736.192393,86.737276,125.932608,55.239385
8,2020-05-01 00:00:00+00:00,8653.0,8653.0,8653.0,8653.0,7842.181898,82.589058,141.930720,75.265514
9,2020-05-01 05:31:00+00:00,8826.5,8826.5,8826.5,8826.5,7945.101892,84.309425,160.151292,94.284856


In [15]:
def add_signals(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['Close','EMA',"MACD","SIGNAL", 'RSI']].to_numpy()[start:end]
    return prices, signal_features

In [16]:
from enum import Enum


# class Actions(Enum):
#     Sell = 0
#     Buy = 1
#     # Hold = 2

from gym import spaces
class Cypto_Env(StocksEnv):
    _process_data = add_signals
    # StocksEnv.action_space = spaces.Discrete(len(Actions))

    

    
env2 = Cypto_Env(df=df, window_size=window_size, frame_bound=(lower_bound,upper_bound))
df.head()

,date,Open,High,Low,Close,EMA,RSI,MACD,SIGNAL
0,2020-04-08 08:09:00+00:00,7314.0,7314.0,7314.0,7314.0,7314.000000,0.000000,0.000000,0.000000
1,2020-04-14 06:31:00+00:00,7101.5,7101.5,7101.5,7101.5,7207.218750,0.000000,-4.767628,-2.648682
2,2020-04-14 07:55:00+00:00,7100.0,7100.0,7100.0,7100.0,7171.121178,0.000000,-6.121420,-4.071935
3,2020-04-22 23:29:00+00:00,7118.5,7118.5,7118.5,7118.5,7157.767900,9.107921,-5.919194,-4.697701
4,2020-04-28 12:28:00+00:00,7748.5,7748.5,7748.5,7748.5,7278.289003,79.058101,19.384455,2.466196


In [17]:
env2.signal_features

array([[ 2.11000000e+04,  2.10296065e+04, -1.97572373e+00,
        -2.41836444e+00,  5.08028470e+01],
       [ 2.11140000e+04,  2.10304463e+04, -4.93423991e-01,
        -2.03337635e+00,  6.29200696e+01],
       [ 2.11190000e+04,  2.10313274e+04,  1.07240542e+00,
        -1.41221999e+00,  6.61287130e+01],
       [ 2.11240000e+04,  2.10322495e+04,  2.68583410e+00,
        -5.92609175e-01,  6.90160898e+01],
       [ 2.11250000e+04,  2.10331724e+04,  3.99908068e+00,
         3.25728796e-01,  6.95747157e+01],
       [ 2.11340000e+04,  2.10341757e+04,  5.70035273e+00,
         1.40065358e+00,  7.41005850e+01],
       [ 2.11250000e+04,  2.10350794e+04,  6.25034817e+00,
         2.37059250e+00,  6.38690153e+01],
       [ 2.11230000e+04,  2.10359542e+04,  6.45048308e+00,
         3.18657062e+00,  6.18260350e+01],
       [ 2.11120000e+04,  2.10367109e+04,  5.65628129e+00,
         3.68051275e+00,  5.19781653e+01],
       [ 2.11110000e+04,  2.10374501e+04,  4.88981228e+00,
         3.92237266e+00

In [18]:
df.head()

,date,Open,High,Low,Close,EMA,RSI,MACD,SIGNAL
0,2020-04-08 08:09:00+00:00,7314.0,7314.0,7314.0,7314.0,7314.000000,0.000000,0.000000,0.000000
1,2020-04-14 06:31:00+00:00,7101.5,7101.5,7101.5,7101.5,7207.218750,0.000000,-4.767628,-2.648682
2,2020-04-14 07:55:00+00:00,7100.0,7100.0,7100.0,7100.0,7171.121178,0.000000,-6.121420,-4.071935
3,2020-04-22 23:29:00+00:00,7118.5,7118.5,7118.5,7118.5,7157.767900,9.107921,-5.919194,-4.697701
4,2020-04-28 12:28:00+00:00,7748.5,7748.5,7748.5,7748.5,7278.289003,79.058101,19.384455,2.466196


In [19]:

 
env_maker = lambda: env2
env = DummyVecEnv([env_maker])

In [40]:
from config import LOG_PATH, MODEL_PATH
from datetime import datetime
import os
 
model = A2C('MlpPolicy', env, verbose=1, tensorboard_log=LOG_PATH, device="cuda") 

  
# model = A2C.load(path=os.path.join("data","model","2022-01-11-19-26-1.zip"),env=env, device="auto")
name = str(datetime.now().strftime("%Y-%d-%m-%H-%M") )
print(name)
episodes = 5
step_count = episodes
for i in range (episodes):
    model = model.learn(total_timesteps=step_count, tb_log_name=f"A2C-{name}"
    )
    print(i)
# #     model.save(f"{os.path.join(MODEL_PATH , name)}-{i + 1}.zip")

------------------------------------
| time/                 |          |
|    fps                | 463      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.686   |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 11699    |
|    policy_loss        | -7.67    |
|    value_loss         | 293      |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 450      |
|    iterations         | 200      |
|    time_elapsed       | 2        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -0.691   |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 11799    |
|    policy_loss        | -0.0165  |
|    value_loss         | 0.00058  |
-

In [21]:
from datetime import timedelta
from collections import deque 
from trader import trader_agent
import time
import os
trade_me = trader_agent()
prev_min = 0
from IPython.display import clear_output
print(len(df)-lower_bound,len(df))

1
100 581999


In [22]:
# obs = env.reset()
# env2.frame_bound

# test = env2.signal_features[window_size - lower_bound:upper_bound]
# last_test = test[-window_size:]
# last_test = last_test[np.newaxis, ...]


# action = model.predict(last_test)

# print(action)
# print(obs)
# print(last_test)
# print(obs == last_test)
# # print(test[0:5])

In [23]:
# # model.load(os.path.join(MODEL_PATH, "2022-01-11-21-49-18.zip") , env = env)
# env = Cypto_Env(df=df, window_size=window_size, frame_bound=(len(df)-80,len(df)))
# obs = env.reset()
# pos_count = deque(maxlen=60)

# env.trade_fee_bid_percent = 0 # unit
# env.trade_fee_ask_percent = 0 # unit
# while True: 
#     try:
#         obs = obs[np.newaxis, ...]
#         action, _states = model.predict(obs)
#         obs, rewards, done, info = env.step(action)
#         cur_min = datetime.now().minute
#         print(action)
#         clear_output()
#         if done:
#             clear_output()
#             reward = env._total_reward

#             pos_count.append(action)
#             print(pos_count)
#             next_df = cm.historical_data_df()
#             next_df = next_df.loc[-100:]
#             next_df['EMA'] = TA.EMA(next_df, 200)
#             prev_min = cur_min
#             next_df['RSI'] = TA.RSI(next_df)
#             next_df.fillna(0, inplace=True)
#             next_df = next_df.join(TA.MACD(next_df))
#             df = pd.concat([df, next_df])
            
#             print(action[0])

#             if action == 1 or action[0] == 1:
#                 trade_me.buy_position_at_market("BTC/USD")
#                 print("b") 
#                 print(df["Low"].tail(1))
#             if action == 0 or action[0] == 0:
#                 trade_me.sell_position_market("BTCUSD")
#                 print("s")
#                 print(df["Low"].tail(1))
#             plt.figure(figsize=(10,6))
#             plt.cla()
#             env.render_all()
#             plt.show()
#             print("info", info)
#             obs = obs[np.newaxis, ...]
#             action, _states = model.predict(obs)
#             obs = env.reset()
#             env.df = df
#             env.frame_bound=(len(df)-lower_bound,len(df))
#             td = timedelta(seconds= 60 - datetime.now().second) 

#             print("waiting for the remainder of the minute")
#             time.sleep(td.total_seconds())
#             env._total_reward = reward
#     except Exception as e:
#         print(e)
#         break


# cur_min = datetime.now().minute

# td = timedelta(seconds= 60 - datetime.now().second) 
# # while True:
# print(td.total_seconds())
#     # if cur_min > prev_min:
#     #     prev_min = cur_min

In [24]:
# plt.figure(figsize=(15,6))
# plt.cla()
# env.render_all()
# plt.show()
# model = model.load(os.path.join(MODEL_PATH, "1667475168.276972.zip"))

In [ ]:
past_trades = deque(maxlen=60)
cur_min = datetime.now().minute
prev_min = 0
test_env = Cypto_Env(df=df, window_size=window_size, frame_bound=(lower_bound,upper_bound))
wait_time = 1
while True:
    try:
        cur_min = datetime.now().minute
        if cur_min > prev_min or cur_min == 1:
            cur_min = datetime.now().minute
            prev_min = cur_min + wait_time - 1
            df = cm.historical_data_df()
            df = df.loc[-100:]
            df['EMA'] = TA.EMA(df, 200)
            df['RSI'] = TA.RSI(df)
            df.fillna(0, inplace=True)
            df = df.join(TA.MACD(df))

            test_env = Cypto_Env(df=df, window_size=window_size, frame_bound=(window_size,upper_bound))
            print(test_env._total_reward)
            def return_obs(df):
                start = len(df) - window_size
                end = len(df)
                signal_features = df.loc[:, ['Low','EMA',"MACD","SIGNAL", 'RSI']].to_numpy()[start:end]
                return signal_features
            
            obs = return_obs(df)
            action = model.predict(obs)
            print("made a prediction")
            print(action)
            if action == 1 or action[0] == 1:
                trade_me.buy_position_at_market("BTC/USD")
                print("b") 
            if action == 0 or action[0] == 0:
                trade_me.sell_position_market("BTCUSD")
                print("s")
            past_trades.append([action, ("price: ", df["Low"].iloc[-1]), ("time: ", datetime.now())])
            for episode in range(episodes):
                model = model.learn(total_timesteps=step_count, tb_log_name= f"A2C-{str(time.time())}")
                model.save(os.path.join(MODEL_PATH, str(time.time()) + ".zip"))
                clear_output()
                print("past trades: ", list(past_trades))
    except Exception as e:
        print(e)
        for episode in range(episodes):
            model = model.learn(total_timesteps=step_count, tb_log_name= f"A2C-{str(time.time())}")
            model.save(os.path.join(MODEL_PATH, str(time.time()) + ".zip"))
            print("past trades: ", list(past_trades))



past trades:  [[(array(1, dtype=int64), None), ('price: ', 21380.0), ('time: ', datetime.datetime(2022, 11, 4, 22, 16, 0, 751948))], [(array(1, dtype=int64), None), ('price: ', 21383.0), ('time: ', datetime.datetime(2022, 11, 4, 22, 17, 1, 139424))], [(array(1, dtype=int64), None), ('price: ', 21374.0), ('time: ', datetime.datetime(2022, 11, 4, 22, 18, 1, 82857))], [(array(0, dtype=int64), None), ('price: ', 21374.0), ('time: ', datetime.datetime(2022, 11, 4, 22, 19, 0, 909654))], [(array(1, dtype=int64), None), ('price: ', 21367.0), ('time: ', datetime.datetime(2022, 11, 4, 22, 21, 0, 740382))], [(array(1, dtype=int64), None), ('price: ', 21371.0), ('time: ', datetime.datetime(2022, 11, 4, 22, 23, 1, 216031))], [(array(1, dtype=int64), None), ('price: ', 21371.0), ('time: ', datetime.datetime(2022, 11, 4, 22, 24, 0, 825148))], [(array(1, dtype=int64), None), ('price: ', 21370.0), ('time: ', datetime.datetime(2022, 11, 4, 22, 25, 0, 902202))], [(array(1, dtype=int64), None), ('price: '

KeyboardInterrupt: 

In [ ]:
model.save(os.path.join(MODEL_PATH, str(time.time()) + ".zip"))